# Advanced Mesh Generation

## Structured Quad Meshes
The triangular meshes generated until now were _unstructured_, that is, they did not have any uniform pattern. Sometimes it is desirable to generate a uniform pattern, in which case we use _structured_ meshes.

To deal with eddy currents in our computation of magnetic fields, we wish to use quad meshes (specifically, using rectangular elements) for the conductive domains, and unstructured triangle meshes for the non-conducting domains. In this part, we focus on generating structured quad meshes with a controllable distribution of nodes.

### Transfinite Meshing
To generate structured meshes, we need some way to impose the desired structure. In Gmsh, we do this by specifying transfinite meshing constraints (`numNodes` nodes distributed according to `meshType` and `coef`) on the curves of the geometry.
```julia
gmsh.model.mesh.setTransfiniteCurve(tag, numNodes, meshType, coef)
```
Supported types are "Progression" (geometrical with power `coef`), "Bump" (refinement toward both extremities) and "Beta" (beta law). By using suitable number of nodes, mesh type, and coefficient, we can obtain the desired meshing detail.

If we desire a surface to have a structured mesh, all of its curves should have defined transfinite meshing constraints, and the surface itself must be transfinite.
```julia
gmsh.model.mesh.setTransfiniteSurface(tag)
```


### Triangle to Quad Meshes
By default, Gmsh meshes using triangular elements. We can construct a quad mesh from these triangular meshes using recombination, which turns two triangles into a quad element. Recombination is automatically done after meshing if we use `setRecombine`.
```julia
gmsh.model.mesh.setRecombine(dim, tag)
```
Currently only entities of dimension 2 (to recombine triangles into quadrangles) are supported.

### Examples

In [1]:
using gmsh

In [ ]:
gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 1)

### Unit Square
Here, we aim to generate a quad mesh on the unit square with a finer mesh on the left side. To do so, we define a Progression, which increases the density of nodes on one side of the curve.

In [ ]:
gmsh.model.add("quad_simple") 
geo  = gmsh.model.geo;
mesh = gmsh.model.mesh;

geo.addPoint(0, 0, 0, 0.05, 1)
geo.addPoint(1, 0, 0, 0.05, 2)
geo.addPoint(1, 1, 0, 0.3, 3)
geo.addPoint(0, 1, 0, 0.3, 4)

geo.addLine(1, 2, 1)
geo.addLine(2, 3, 2)
geo.addLine(3, 4, 3)
geo.addLine(4, 1, 4)

geo.addCurveLoop([1,2,3,4], 1)
geo.addPlaneSurface([1], 1)

geo.synchronize()

# Set the transfinite interpolations to generate a structured mesh
mesh.setTransfiniteCurve(1, 30, "Progression", 1.1)
mesh.setTransfiniteCurve(3, 30, "Progression", -1.1)   # Negative coefficient because curve 3 is oriented the opposite way
mesh.setTransfiniteCurve(2, 7)
mesh.setTransfiniteCurve(4, 7)
mesh.setTransfiniteSurface(1, "Left")

# Recombine triangles into quad elements
mesh.setRecombine(2, 1)

mesh.generate(2)

gmsh.write("geo_files/square_quad.msh")

In [ ]:
gmsh.fltk.run()

![Unit Square with Quad Mesh](figures/square_quad.png)

### Hybrid Meshes
By using `setRecombine` only on a part of the surfaces, we can effortlessly create hybrid meshes with quad and triangle elements. This is also possible if transfinite interpolations are _not_ used. In this case, a completely unstructured hybrid mesh will result.

This is illustrated on a conductive magnetic core (e.g., Silicon steel or similar), which is meshed using structured quads to aid in eddy current computations. The computation domain and air inside the core are non-conductive, and hence meshed with unstructured triangles.

In [ ]:
gmsh.model.add("quad_simple") 
geo  = gmsh.model.geo;
mesh = gmsh.model.mesh;

# Points
geo.addPoint(0, 0, 0, 0.1, 1)
geo.addPoint(1, 0, 0, 0.1, 2)
geo.addPoint(1, 1, 0, 0.1, 3)
geo.addPoint(0, 1, 0, 0.1, 4)

geo.addPoint(0.2, 0.2, 0, 0.1, 5)
geo.addPoint(0.35, 0.2, 0, 0.1, 6)
geo.addPoint(0.65, 0.2, 0, 0.1, 7)
geo.addPoint(0.8, 0.2, 0, 0.1, 8)
geo.addPoint(0.8, 0.35, 0, 0.1, 9)
geo.addPoint(0.8, 0.65, 0, 0.1, 10)
geo.addPoint(0.8, 0.8, 0, 0.1, 11)
geo.addPoint(0.65, 0.8, 0, 0.1, 12)
geo.addPoint(0.35, 0.8, 0, 0.1, 13)
geo.addPoint(0.2, 0.8, 0, 0.1, 14)
geo.addPoint(0.2, 0.65, 0, 0.1, 15)
geo.addPoint(0.2, 0.35, 0, 0.1, 16)

geo.addPoint(0.35, 0.35, 0, 0.1, 17)
geo.addPoint(0.65, 0.35, 0, 0.1, 18)
geo.addPoint(0.65, 0.65, 0, 0.1, 19)
geo.addPoint(0.35, 0.65, 0, 0.1, 20)

# Lines
geo.addLine(1, 2, 1)
geo.addLine(2, 3, 2)
geo.addLine(3, 4, 3)
geo.addLine(4, 1, 4)

geo.addLine(5, 6, 5)
geo.addLine(6, 7, 6)
geo.addLine(7, 8, 7)
geo.addLine(8, 9, 8)
geo.addLine(9, 10, 9)
geo.addLine(10, 11, 10)
geo.addLine(11, 12, 11)
geo.addLine(12, 13, 12)
geo.addLine(13, 14, 13)
geo.addLine(14, 15, 14)
geo.addLine(15, 16, 15)
geo.addLine(16, 5, 16)

geo.addLine(6, 17, 17)
geo.addLine(7, 18, 18)
geo.addLine(9, 18, 19)
geo.addLine(10, 19, 20)
geo.addLine(12, 19, 21)
geo.addLine(13, 20, 22)
geo.addLine(15, 20, 23)
geo.addLine(16, 17, 24)

geo.addLine(17, 18, 25)
geo.addLine(18, 19, 26)
geo.addLine(19, 20, 27)
geo.addLine(20, 17, 28)

# Surfaces
geo.addCurveLoop([1,2,3,4], 1)
geo.addCurveLoop([5,6,7,8,9,10,11,12,13,14,15,16], 2)
geo.addCurveLoop([5,17,24,16], 3, true)
geo.addCurveLoop([6,18,25,17], 4, true)
geo.addCurveLoop([7,8,19,18], 5, true)
geo.addCurveLoop([9,20,26,19], 6, true)
geo.addCurveLoop([10,11,21,20], 7, true)
geo.addCurveLoop([12,22,27,21], 8, true)
geo.addCurveLoop([13,14,23,22], 9, true)
geo.addCurveLoop([15,24,28,23], 10, true)
geo.addCurveLoop([25,26,27,28], 11, true)

geo.addPlaneSurface([1, 2], 1)
geo.addPlaneSurface([3], 2)
geo.addPlaneSurface([4], 3)
geo.addPlaneSurface([5], 4)
geo.addPlaneSurface([6], 5)
geo.addPlaneSurface([7], 6)
geo.addPlaneSurface([8], 7)
geo.addPlaneSurface([9], 8)
geo.addPlaneSurface([10], 9)
geo.addPlaneSurface([11], 10)

geo.synchronize()

# Set the transfinite interpolations to generate a structured mesh
N  = 20;
N2 = 10;
mesh.setTransfiniteCurve(17, N, "Bump", 0.05)
mesh.setTransfiniteCurve(24, N, "Bump", 0.05)
mesh.setTransfiniteCurve( 5, N)
mesh.setTransfiniteCurve(16, N)
mesh.setTransfiniteSurface(2, "Left")

mesh.setTransfiniteCurve(18, N, "Bump", 0.05)
mesh.setTransfiniteCurve(19, N, "Bump", 0.05)
mesh.setTransfiniteCurve( 7, N)
mesh.setTransfiniteCurve( 8, N)
mesh.setTransfiniteSurface(4, "Left")

mesh.setTransfiniteCurve(20, N, "Bump", 0.05)
mesh.setTransfiniteCurve(21, N, "Bump", 0.05)
mesh.setTransfiniteCurve(10, N)
mesh.setTransfiniteCurve(11, N)
mesh.setTransfiniteSurface(6, "Left")

mesh.setTransfiniteCurve(22, N, "Bump", 0.05)
mesh.setTransfiniteCurve(23, N, "Bump", 0.05)
mesh.setTransfiniteCurve(14, N)
mesh.setTransfiniteCurve(13, N)
mesh.setTransfiniteSurface(8, "Left")

mesh.setTransfiniteCurve( 6, N2)
mesh.setTransfiniteCurve(25, N2)
mesh.setTransfiniteSurface(3, "Left")

mesh.setTransfiniteCurve( 9, N2)
mesh.setTransfiniteCurve(26, N2)
mesh.setTransfiniteSurface(5, "Left")

mesh.setTransfiniteCurve(12, N2)
mesh.setTransfiniteCurve(27, N2)
mesh.setTransfiniteSurface(7, "Left")

mesh.setTransfiniteCurve(15, N2)
mesh.setTransfiniteCurve(28, N2)
mesh.setTransfiniteSurface(9, "Left")

# Recombine triangles into quad elements
mesh.setRecombine(2, 2)
mesh.setRecombine(2, 3)
mesh.setRecombine(2, 4)
mesh.setRecombine(2, 5)
mesh.setRecombine(2, 6)
mesh.setRecombine(2, 7)
mesh.setRecombine(2, 8)
mesh.setRecombine(2, 9)

mesh.generate(2)

gmsh.write("geo_files/hybrid.msh")

In [ ]:
gmsh.fltk.run()

![Hybrid Mesh](figures/hybrid_mesh.png)

## Second-order Triangular Elements

In [ ]:
ri = 25;
ro = 51;

mshd_out = ro / 3; # Mesh density at outer boundary
mshd_in  = ri / 3; # Mesh density at inner boundary

In [ ]:
gmsh.initialize()

In [ ]:
gmsh.model.add("coax_cable")
geo = gmsh.model.geo;

## Points
geo.addPoint(0, 0, 0, mshd_out, 1)
geo.addPoint(ri, 0, 0, mshd_in, 2)
geo.addPoint(ro, 0, 0, mshd_out, 3)
geo.addPoint(-ri, 0, 0, mshd_in, 4)
geo.addPoint(-ro, 0, 0, mshd_out, 5)

## Curves
geo.addCircleArc(2, 1, 4, 1)
geo.addCircleArc(4, 1, 2, 2)
geo.addCircleArc(3, 1, 5, 3)
geo.addCircleArc(5, 1, 3, 4)

## Surfaces
geo.addCurveLoop([1, 2], 1)
geo.addCurveLoop([3, 4], 2)

geo.addPlaneSurface([2, 1], 1)
geo.addPlaneSurface([1], 2)

## Define domains
geo.addPhysicalGroup(2, [1], 1)
geo.addPhysicalGroup(2, [2], 2)

# Generate mesh
geo.synchronize();
gmsh.model.mesh.generate(2)
gmsh.model.mesh.setOrder(2)

gmsh.write("geo_files/coaxial_cable2.msh")

In [ ]:
gmsh.fltk.run()